In [64]:
#coding:utf-8
import pandas as pd
import time
import gc
from sklearn.metrics import log_loss
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    data['minute'] = data['context_timestamp_tmp'].dt.minute
    data['day'] = data['context_timestamp_tmp'].dt.day
    data['halfhour'] = (data['hour']+1)*3+(data['minute']/20).astype('int')
    del data['context_timestamp_tmp']
    del data['minute']
    del data['week']
    del data['hour']
    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']
    del data['shop_score_service']
    del data['shop_score_description']
    del data['shop_review_positive_rate']
    del data['shop_score_delivery']
    return data

In [65]:
print('train')
train = pd.read_table('train.txt',sep=' ',index_col=False)
dataa = pd.read_csv('train_all.csv',index_col=False)

train


## 特征选择

In [66]:
dataa.columns

Index(['instance_id', 'hour_num', 'item_click', 'shop_score_service_num',
       'shop_score_description_num', 'shop_review_positive_rate_num',
       'shop_score_delivery_num', 'user_click', 'property_0_click',
       'property_1_click', 'property_2_click', 'shop_click', 'brand_click',
       'category_1_pvnum', 'category_1_first', 'category_1_last',
       'property_0_pvnum', 'property_0_first', 'property_0_last',
       'property_1_pvnum', 'property_1_first', 'property_1_last',
       'item_id_pvnum', 'item_id_first', 'item_id_last', 'occupation_click',
       'user_gender_click', 'day_click'],
      dtype='object')

In [67]:
tezheng = ['instance_id', 'hour_num', 'item_click', 'shop_score_service_num',
       'shop_score_description_num', 'shop_review_positive_rate_num',
       'shop_score_delivery_num', 'user_click', 'property_0_click',
       'property_1_click', 'shop_click', 'brand_click',
       'category_1_pvnum', 'category_1_first', 'category_1_last',
       'property_0_pvnum', 'property_0_first', 'property_0_last',
       'property_1_pvnum', 'property_1_first', 'property_1_last',
       'item_id_pvnum', 'item_id_first', 'item_id_last', 'occupation_click',
       'user_gender_click', 'day_click']

In [68]:
dataa = dataa[tezheng]

In [69]:

train = pd.merge(train,dataa,on=['instance_id'],how='left')
del(dataa)
gc.collect()
train = pre_process(train)

预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [70]:
all_data = train.copy()
print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['day'] == 7]

# 空去 22 号

train = train[train['day'] != 7]

# train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)


all_shape (10432036, 54)
2018-09-07 11:59:59
(9354861, 54)
(1077175, 54)


In [71]:

print('test')
test_a = pd.read_table('test.txt',sep=' ',index_col=False) 
data2a = pd.read_csv('test_all.csv',index_col=False) 

# del train[['user_id', 'category_1', 'property_2', 'property_0']]
# del val[['user_id', 'category_1', 'property_2', 'property_0']]
# del test_a[['user_id', 'category_1', 'property_2', 'property_0']]
# 这里是增加的内容


test


## 特征选择

In [72]:
data2a = data2a[tezheng]

In [73]:
test_a.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description'],
      dtype='object')

In [74]:
test_a = pd.merge(test_a,data2a,on=['instance_id'],how='left')
del(data2a)
gc.collect()
print(test_a.shape)
test_a = pre_process(test_a)
del train['item_city_id']
del val['item_city_id']
del test_a['item_city_id']
del train['shop_id']
del val['shop_id']
del test_a['shop_id']
del train['context_id']
del val['context_id']
del test_a['context_id']
del train['item_brand_id']
del val['item_brand_id']
del test_a['item_brand_id']
# del train['category_0']
# del val['category_0']
# del test_a['category_0']
# del train['category_2']
# del val['category_2']
# del test_a['category_2']
# del train['property_1']
# del val['property_1']
# del test_a['property_1']
# del train['property_2']
# del val['property_2']
# del test_a['property_2']

(519888, 52)
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [75]:

# import datetime
# def get_count_feat(all_data,data,long=3):
#     end_time = data['context_timestamp'].min()
#     begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
#     all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
#     all_data = all_data[
#         (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
#                     ]
    
#     print(end_time)
#     print(begin_time)
#     print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
#     item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
#     item_count.rename(columns={0:'item_count'},inplace=True)

#     user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
#     user_count.rename(columns={0: 'user_count'}, inplace=True)
#     return user_count,item_count




In [76]:

# train_user_count,train_item_count = get_count_feat(all_data,train,2)

# test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

# val_user_count,val_item_count = get_count_feat(all_data,val,2)
# # print(train.columns)
# train = pd.merge(train,train_user_count,on=['user_id'],how='left')
# train = pd.merge(train,train_item_count,on=['item_id'],how='left')
# train = train.fillna(-1)
# val = pd.merge(val,val_user_count,on=['user_id'],how='left')
# val = pd.merge(val,val_item_count,on=['item_id'],how='left')
# val = val.fillna(-1)
# test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
# test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
# test_a = test_a.fillna(-1)

# 删掉上面增加的用户点击量与商品点击量
# del train['user_count']
# del train['item_count']
# del val['user_count']
# del val['item_count']
# del test_a['user_count']
# del test_a['item_count']

# user_item_count = pd.read_csv


y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')

print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data


(519888, 48)


In [77]:
print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression

baseline ing ... ...


In [78]:

# print('baseline ing ... ...')
# from sklearn.preprocessing import LabelEncoder,OneHotEncoder
# from scipy import sparse
# from sklearn.linear_model import LogisticRegression
# # print(test_a.columns)

# enc = OneHotEncoder()
# lb = LabelEncoder()
# # feat_set = list(test_a.columns)

# del train['user_id']
# del test_a['user_id']
# del val['user_id']
# # del train['item_id']
# # del test_a['item_id']
# # del val['item_id']
# # 8288

# print(train.columns)

# # 自己使用onehot编码
# train['x_train'] = 1
# test_a['x_test'] = 1
# val['x_val'] = 1

# all_data = pd.concat([train,test_a,val])
# all_data = pd.get_dummies(all_data)
# print(all_data.shape)

# X_train = all_data.loc[all_data['x_train'].notnull()]
# X_test = all_data.loc[all_data['x_test'].notnull()]
# X_val = all_data.loc[all_data['x_val'].notnull()]

# del X_train['x_train']
# del X_train['x_test']
# del X_train['x_val']
# del X_test['x_train']
# del X_test['x_test']
# del X_test['x_val']
# del X_val['x_train']
# del X_val['x_test']
# del X_val['x_val']


In [79]:
# LogisticRegression?

In [80]:

# lr = LogisticRegression(C=5.0,penalty='l1')

# lr.fit(X_train, y_train)


In [81]:

# proba_val = lr.predict_proba(X_val)[:,1]
# proba_sub = lr.predict_proba(X_test)[:,1]
# print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
# print(log_loss(y_val,proba_val))
# print()

In [82]:
# category_0, category_1, category_2, property_0, property_1, property_2
del val['category_0']
del val['category_1']
del val['category_2']
del val['property_0']
del val['property_1']
del val['property_2']
# train = pd.DataFrame(train,dtype='float')
del train['category_0']
del train['category_1']
del train['category_2']
del train['property_0']
del train['property_1']
del train['property_2']

del test_a['category_0']
del test_a['category_1']
del test_a['category_2']
del test_a['property_0']
del test_a['property_1']
del test_a['property_2']

In [83]:
train.columns

Index(['item_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level',
       'context_page_id', 'shop_review_num_level', 'shop_star_level',
       'hour_num', 'item_click', 'shop_score_service_num',
       'shop_score_description_num', 'shop_review_positive_rate_num',
       'shop_score_delivery_num', 'user_click', 'property_0_click',
       'property_1_click', 'shop_click', 'brand_click', 'category_1_pvnum',
       'category_1_first', 'category_1_last', 'property_0_pvnum',
       'property_0_first', 'property_0_last', 'property_1_pvnum',
       'property_1_first', 'property_1_last', 'item_id_pvnum', 'item_id_first',
       'item_id_last', 'occupation_click', 'user_gender_click', 'day_click',
       'day', 'halfhour'],
      dtype='object')

In [84]:
def changeNaN(x):
    if x==-1:
        return float("nan") 
    else:
        return x

In [85]:
train['item_price_level'].apply(changeNaN)
train['item_sales_level'].apply(changeNaN)
train['item_collected_level'].apply(changeNaN)
train['item_pv_level'].apply(changeNaN)
train['user_gender_id'].apply(changeNaN)
train['user_age_level'].apply(changeNaN)
train['user_occupation_id'].apply(changeNaN)
train['shop_review_num_level'].apply(changeNaN)
train['shop_star_level'].apply(changeNaN)
pass

In [86]:
val['item_price_level'].apply(changeNaN)
val['item_sales_level'].apply(changeNaN)
val['item_collected_level'].apply(changeNaN)
val['item_pv_level'].apply(changeNaN)
val['user_gender_id'].apply(changeNaN)
val['user_age_level'].apply(changeNaN)
val['user_occupation_id'].apply(changeNaN)
val['shop_review_num_level'].apply(changeNaN)
val['shop_star_level'].apply(changeNaN)
pass

In [87]:
test_a['item_price_level'].apply(changeNaN)
test_a['item_sales_level'].apply(changeNaN)
test_a['item_collected_level'].apply(changeNaN)
test_a['item_pv_level'].apply(changeNaN)
test_a['user_gender_id'].apply(changeNaN)
test_a['user_age_level'].apply(changeNaN)
test_a['user_occupation_id'].apply(changeNaN)
test_a['shop_review_num_level'].apply(changeNaN)
test_a['shop_star_level'].apply(changeNaN)
pass

In [90]:

import lightgbm as lgb
gbm = lgb.LGBMRegressor(objective='binary',
                        num_leaves=64,
                        max_depth = 8,
                        device = "gpu",
                        gpu_platform_id = 0,
                        gpu_device_id = 0,
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.7,
                        reg_alpha = 1
                       )
gbm.fit(train, y_train,
        eval_set=[(val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)



[1]	valid_0's binary_logloss: 0.684332
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.675664
[3]	valid_0's binary_logloss: 0.667088
[4]	valid_0's binary_logloss: 0.658742
[5]	valid_0's binary_logloss: 0.650639
[6]	valid_0's binary_logloss: 0.642608
[7]	valid_0's binary_logloss: 0.634801
[8]	valid_0's binary_logloss: 0.62714
[9]	valid_0's binary_logloss: 0.619581
[10]	valid_0's binary_logloss: 0.612228
[11]	valid_0's binary_logloss: 0.605007
[12]	valid_0's binary_logloss: 0.597896
[13]	valid_0's binary_logloss: 0.590953
[14]	valid_0's binary_logloss: 0.584198
[15]	valid_0's binary_logloss: 0.577552
[16]	valid_0's binary_logloss: 0.570933
[17]	valid_0's binary_logloss: 0.564468
[18]	valid_0's binary_logloss: 0.558095
[19]	valid_0's binary_logloss: 0.55189
[20]	valid_0's binary_logloss: 0.545864
[21]	valid_0's binary_logloss: 0.53987
[22]	valid_0's binary_logloss: 0.533972
[23]	valid_0's binary_logloss: 0.528197
[24]	valid_0's binary_logloss

[203]	valid_0's binary_logloss: 0.191353
[204]	valid_0's binary_logloss: 0.191056
[205]	valid_0's binary_logloss: 0.190765
[206]	valid_0's binary_logloss: 0.190484
[207]	valid_0's binary_logloss: 0.190208
[208]	valid_0's binary_logloss: 0.189945
[209]	valid_0's binary_logloss: 0.189682
[210]	valid_0's binary_logloss: 0.189429
[211]	valid_0's binary_logloss: 0.189174
[212]	valid_0's binary_logloss: 0.188924
[213]	valid_0's binary_logloss: 0.188679
[214]	valid_0's binary_logloss: 0.188443
[215]	valid_0's binary_logloss: 0.188208
[216]	valid_0's binary_logloss: 0.187987
[217]	valid_0's binary_logloss: 0.187771
[218]	valid_0's binary_logloss: 0.187564
[219]	valid_0's binary_logloss: 0.187359
[220]	valid_0's binary_logloss: 0.187151
[221]	valid_0's binary_logloss: 0.186948
[222]	valid_0's binary_logloss: 0.186761
[223]	valid_0's binary_logloss: 0.186579
[224]	valid_0's binary_logloss: 0.18639
[225]	valid_0's binary_logloss: 0.186218
[226]	valid_0's binary_logloss: 0.186038
[227]	valid_0's b

[404]	valid_0's binary_logloss: 0.190738
[405]	valid_0's binary_logloss: 0.190839
[406]	valid_0's binary_logloss: 0.190906
[407]	valid_0's binary_logloss: 0.191006
[408]	valid_0's binary_logloss: 0.191105
[409]	valid_0's binary_logloss: 0.191184
[410]	valid_0's binary_logloss: 0.19128
[411]	valid_0's binary_logloss: 0.191399
[412]	valid_0's binary_logloss: 0.191484
[413]	valid_0's binary_logloss: 0.19157
[414]	valid_0's binary_logloss: 0.191635
[415]	valid_0's binary_logloss: 0.191738
[416]	valid_0's binary_logloss: 0.191823
[417]	valid_0's binary_logloss: 0.191938
[418]	valid_0's binary_logloss: 0.192009
[419]	valid_0's binary_logloss: 0.192105
[420]	valid_0's binary_logloss: 0.192178
[421]	valid_0's binary_logloss: 0.192273
[422]	valid_0's binary_logloss: 0.192347
[423]	valid_0's binary_logloss: 0.192407
[424]	valid_0's binary_logloss: 0.192513
[425]	valid_0's binary_logloss: 0.192628
[426]	valid_0's binary_logloss: 0.192743
[427]	valid_0's binary_logloss: 0.192823
Early stopping, be

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
       device='gpu', gpu_device_id=0, gpu_platform_id=0,
       learning_rate=0.01, max_depth=8, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=-1, num_leaves=64, objective='binary', random_state=None,
       reg_alpha=1, reg_lambda=0.0, silent=True, subsample=0.7,
       subsample_for_bin=200000, subsample_freq=1)

In [91]:
print('Start predicting...')
# predict
# y_pred_1 = gbm.predict(val, num_iteration=gbm.best_iteration_ )
y_sub_1 = gbm.predict(test_a, num_iteration=gbm.best_iteration_ )
print(log_loss(y_train,gbm.predict(train)))
# print(log_loss(y_val,y_pred_1))

# print(log_loss(y_val,(y_pred_1 + proba_val)/2))#0.0802392979662

Start predicting...
0.0740458167255


In [33]:

# import numpy as np
# bes = 1
# for i in np.arange(0,1,0.01):
#     if(log_loss(y_val,(i * y_pred_1 + (1-i) * proba_val)) < bes):
#         bes = log_loss(y_val,(y_pred_1 + proba_val)/2)
# print(bes)#0.083131236937

# xx_analyse = pd.DataFrame()
# xx_analyse['ture'] = list(y_val)
# xx_analyse['pre'] = list(proba_val)
# xx_analyse['pre_1'] = list(y_pred_1)
# xx_analyse.to_csv('temp.csv',index=False)
#
# sub = pd.DataFrame()
# sub['instance_id'] = list(test_index)
# sub['predicted_score'] = list(y_sub_1)
# sub.to_csv('feather/20180327.txt',sep=" ",index=False)

In [34]:
y_sub_1.shape

(519888,)

In [35]:
y_sub_1

array([ 0.03500097,  0.0337831 ,  0.04590906, ...,  0.03276625,
        0.05857461,  0.04220761])

In [36]:
sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180430.txt',sep=" ",index=False)